In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import time

# Import log to merge metadata later
log = pd.read_csv('download_log.csv')
display(log.head())

In [ ]:
# Loop through the HTML files in the SEC_data directory and add text to a csv file
save_directory = os.path.join("..", "SEC_data")
data = []

for file_name in os.listdir(save_directory):
    file_path = os.path.join(save_directory, file_name)
    # Read the content of the file
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
        # Convert HTML to plain text
        soup = BeautifulSoup(html_content, 'html.parser')
        text = soup.get_text(separator='\n')
        # Extract information from the file name
        access_number, ticker, form, date = file_name.split('_')
        date = date.replace('.html', '')
        # Add the content to the data list
        data.append({
            'AccessionNumber': access_number,
            'Ticker': ticker,
            'Date': date,
            'Form': form,
            'Text': text
        })

# Create a DataFrame from the data list
df = pd.DataFrame(data)
display(df.head())

In [ ]:
# Add metadata from log
